In [1]:
# Getting more statistics about shapes and traces count for each bus
# - exploring the first quantile for working days
# - exploring the second quantile for holiday and weekends

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
1,application_1609519824222_0002,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
from pyspark import SparkConf
from pyspark import SparkContext

spark_conf = (SparkConf().set("spark.speculation", "false"))
sc = SparkContext.getOrCreate(conf = spark_conf)

# spark = sparkSession
hadoop_conf = spark._jsc.hadoopConfiguration()
hadoop_conf.set("fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
hadoop_conf.set("mapreduce.fileoutputcommitter.algorithm.version","2")

# installing required packages for this notebook session
sc.install_pypi_package("boto3")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

  Using cached https://files.pythonhosted.org/packages/8f/4a/16ffdfc33d93f02604ae9ed1ddb6369030b6f61b583f31dc84e0d0da05c1/botocore-1.19.47-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/69/79/e6afb3d8b0b4e96cefbdc690f741d7dd24547ff1f94240c997a26fa908d3/s3transfer-0.3.3-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/d4/70/d60450c3dd48ef87586924207ae8907090de0b306af2bce5d134d78615cb/python_dateutil-2.8.1-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/f5/71/45d36a8df68f3ebb098d6861b2c017f3d094538c0fb98fa61d4dc43e69b9/urllib3-1.26.2-py2.py3-none-any.whl

In [ ]:
import pyspark.sql.functions as f

# files' statistics header
csv_out_shape_count = "day,total_size,shape_seq_count_mean,shape_seq_count_min,shape_seq_count_max,shape_seq_count_stddev,shape_seq_count_quantile_25,shape_seq_count_quantile_50,shape_seq_count_quantile_75\n"
csv_out_trace_count = "day,total_size,traces_count_mean,traces_count_min,traces_count_max,traces_count_stddev,traces_count_quantile_25,traces_count_quantile_50,traces_count_quantile_75\n"

days_second_quantile_50 = set([4,11,12,18,26,17])
days_first_quantile_25 = set(range(1,32)) - days_second_quantile_50

traces_count_stats  = spark.read.csv("s3://mobility-traces-sp/statistics/exploring-data/7-traces_per_bus.csv",header=True)

for day in days_first_quantile_25:
    
    # reading map matching files
    traces = spark.read.parquet("s3://mobility-traces-sp/processed-data-avl-date/8-map-matching/MO_1510"+ str(day) + "/")
    
    # getting the number of registers
    total_size = traces.count()

    # counting the number of different shape sequences for a bus in a day
    shape_count = traces.groupBy("id_avl","line_id").agg(f.countDistinct("min_shape_sequence").alias("shape_seq_count"))
    
    # the limits of the first quantile of 25%
    shape_count = shape_count.filter("shape_seq_count < 101")

    # getting some statistics
    stats_shape_seq_count = shape_count.agg(f.mean('shape_seq_count').alias('mean'),
                       f.min('shape_seq_count').alias('min'),
                       f.max('shape_seq_count').alias('max'),
                       f.stddev('shape_seq_count').alias("stddev")).collect()
    
    # getting the quantiles
    shape_seq_count_quantile = shape_count.approxQuantile("shape_seq_count", [0.25,0.5,0.75], 0.0001)
    
    
    csv_out_shape_count += "MO_1510" + str(day) +","+ str(total_size) +","+ str(stats_shape_seq_count[0]["mean"]) +","+ str(stats_shape_seq_count[0]["min"]) + "," + str(stats_shape_seq_count[0]["max"]) + "," + str(stats_shape_seq_count[0]["stddev"]) + "," +str(shape_seq_count_quantile[0]) + "," +str(shape_seq_count_quantile[1]) +"," +str(shape_seq_count_quantile[2]) + "\n"


import boto3   
s3 = boto3.client('s3')

# writing results in S3
s3.put_object(Body=bytes(csv_out_shape_count,"utf-8"), Bucket='mobility-traces-sp', Key='statistics/exploring-data/8-shapes_seq_per_bus_25_quantile.csv')


for day in range(1,32):
    # reading map matching files
    traces = spark.read.parquet("s3://mobility-traces-sp/processed-data-avl-date/8-map-matching/MO_1510"+ str(day) + "/")
    
    # getting the number of registers
    total_size = traces.count()
    
    # counting the number of traces for a bus in a day
    trace_count = traces.groupBy("id_avl","line_id").agg(f.count("id_avl").alias("traces_count_per_bus"))
    
    # limit of the quantile
    limit_quantile_25 = traces_count_stats.filter("day == 'MO_1510" + str(day)+"'").collect()[0]["traces_count_quantile_25"]

    trace_count = trace_count.filter("traces_count_per_bus < " + str(limit_quantile_25))
    
    # getting some statistics
    stats_trace_count = trace_count.agg(f.mean('traces_count_per_bus').alias('mean'),
                       f.min('traces_count_per_bus').alias('min'),
                       f.max('traces_count_per_bus').alias('max'),
                       f.stddev('traces_count_per_bus').alias("stddev")).collect()
    
    # getting the quantiles
    trace_count_quantile = trace_count.approxQuantile("traces_count_per_bus", [0.25,0.5,0.75], 0.0001)
    
    csv_out_trace_count += "MO_1510" + str(day) +","+ str(total_size) +","+ str(stats_trace_count[0]["mean"]) +","+ str(stats_trace_count[0]["min"]) + "," + str(stats_trace_count[0]["max"]) + "," + str(stats_trace_count[0]["stddev"]) + "," +str(trace_count_quantile[0]) + "," +str(trace_count_quantile[1]) +"," +str(trace_count_quantile[2]) +"\n"

    
s3.put_object(Body=bytes(csv_out_trace_count,"utf-8"), Bucket='mobility-traces-sp', Key='statistics/exploring-data/7-traces_per_bus_25_quantile.csv')    

In [ ]:
traces_count_stats  = spark.read.csv("s3://mobility-traces-sp/statistics/exploring-data/7-traces_per_bus_25_quantile.csv",header=True)
csv_out_trace_count = "day,total_size,traces_count_mean,traces_count_min,traces_count_max,traces_count_stddev,traces_count_quantile_25,traces_count_quantile_50,traces_count_quantile_75\n"

for day in range(1,32):
    # reading map matching files
    traces = spark.read.parquet("s3://mobility-traces-sp/processed-data-avl-date/8-map-matching/MO_1510"+ str(day) + "/")
    
    # getting the number of registers
    total_size = traces.count()
    
    # counting the number of traces for a bus in a day
    trace_count = traces.groupBy("id_avl","line_id").agg(f.count("id_avl").alias("traces_count_per_bus"))
    
    # limit of the quantile
    limit_quantile_25 = traces_count_stats.filter("day == 'MO_1510" + str(day)+"'").collect()[0]["traces_count_quantile_25"]
    trace_count = trace_count.filter("traces_count_per_bus < " + str(limit_quantile_25))
    # getting some statistics
    stats_trace_count = trace_count.agg(f.mean('traces_count_per_bus').alias('mean'),
                       f.min('traces_count_per_bus').alias('min'),
                       f.max('traces_count_per_bus').alias('max'),
                       f.stddev('traces_count_per_bus').alias("stddev")).collect()
    # getting the quantiles
    trace_count_quantile = trace_count.approxQuantile("traces_count_per_bus", [0.25,0.5,0.75], 0.0001)
    trace_count_quantile
    csv_out_trace_count += "MO_1510" + str(day) +","+ str(total_size) +","+ str(stats_trace_count[0]["mean"]) +","+ str(stats_trace_count[0]["min"]) + "," + str(stats_trace_count[0]["max"]) + "," + str(stats_trace_count[0]["stddev"]) + "," +str(trace_count_quantile[0]) + "," +str(trace_count_quantile[1]) +"," +str(trace_count_quantile[2]) +"\n"

    
s3.put_object(Body=bytes(csv_out_trace_count,"utf-8"), Bucket='mobility-traces-sp', Key='statistics/exploring-data/7-traces_per_bus_25_quantile_2_times.csv')    